In [40]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import logging
import pandas as pd
import pandahouse
import os
import datetime as dt

#sns.set(rc={'figure.figsize':(19, 13), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12,'axes.titlepad': 30 })

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220220'
}

In [41]:
bot = telegram.Bot(token = '5291500178:AAFQqAVPSG4ad90QdyE-IaZH-Ai8ow2oxFs')
#updates = bot.getUpdates()
#print(updates[-1])

In [42]:
#chat_id = 1295860693
chat_id = -1001735251250

In [43]:
q = 'select toDate(time) as date, user_id, action  from simulator_20220220.feed_actions where toDate(time) > today()-8 and toDate(time) < today()'

df_feed = pandahouse.read_clickhouse(q, connection=connection)

In [44]:
q = 'select toDate(time) as date, user_id, reciever_id  from simulator_20220220.message_actions where toDate(time) > today()-8 and toDate(time) < today()'

df_mess = pandahouse.read_clickhouse(q, connection=connection)

In [19]:
# FEED AND MESSAGE REPORT

In [20]:
# FEED METRICS

In [45]:
DAU_FEED = df_feed.groupby('date', as_index = False).user_id.nunique().rename(columns={'user_id':'DAU_FEED'})
Likes = df_feed.loc[df_feed.action == 'like'].groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'likes'})
Views = df_feed.loc[df_feed.action == 'view'].groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'views'})
report_metrics = DAU_FEED.merge(Likes).merge(Views)
report_metrics['LPU'] = round(report_metrics.likes/report_metrics.DAU_FEED,2)
report_metrics['VPU'] = round(report_metrics.views/report_metrics.DAU_FEED,2)
report_metrics['CTR_%'] = round(report_metrics.likes*100/report_metrics.views,2)
report_metrics['actions'] = report_metrics.likes+report_metrics.views

,date,DAU_FEED,likes,views,LPU,VPU,CTR_%,actions
0,2022-03-21,19540,145441,662836,7.44,33.92,21.94,808277
1,2022-03-22,19702,143682,660814,7.29,33.54,21.74,804496
2,2022-03-23,19717,120339,589754,6.10,29.91,20.40,710093
3,2022-03-24,19741,92449,471127,4.68,23.87,19.62,563576
4,2022-03-25,19583,96204,474993,4.91,24.26,20.25,571197
5,2022-03-26,18882,137902,650507,7.30,34.45,21.20,788409
6,2022-03-27,18727,160796,739655,8.59,39.50,21.74,900451


In [22]:
# MESSAGE METRICS

In [50]:
MESSAGE = df_mess.groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'message'})
DAU_MESS = df_mess.groupby('date', as_index=False).user_id.nunique().rename(columns={'user_id':'DAU_MESS'})
DAU_FEEDandMESS = df_feed.merge(df_mess, how = 'inner', on = ['date','user_id'])\
                        .groupby('date', as_index=False)\
                        .user_id.nunique()\
                        .rename(columns={'user_id':'DAU_FEEDandMEES'})
report_metrics = report_metrics.merge(DAU_FEEDandMESS).merge(MESSAGE).merge(DAU_MESS)
report_metrics['MPU']=round(report_metrics.message/report_metrics.DAU_MESS,2)

In [52]:
report_metrics = report_metrics.reindex(columns=['date', 'likes', 'views','CTR_%', 'DAU_FEED', 'DAU_MESS','DAU_FEEDandMEES', 'actions', 'LPU','VPU', 'message','MPU'])

In [54]:
#разворачиваем, чтобы 1 была вчерашняя дата
invers_report_metrics = report_metrics.sort_values('date', ascending=False)
#сохраняем данные за вчерашнюю дату
yesterday_report_metrics = invers_report_metrics.iloc[0, :]
#находим процентное отношение остальных дат к вчерашней
invers_report_metrics.iloc[:, 1:] = (invers_report_metrics.iloc[:, 1:] - invers_report_metrics.iloc[0, 1:].values.squeeze())\
                                        .div(invers_report_metrics.iloc[:, 1:])
#восстанавливаем значения за вчера
for i in range(len(yesterday_report_metrics)):
    invers_report_metrics.iloc[0, i] =yesterday_report_metrics[i]
#переводим в проценты и строки
invers_report_metrics.iloc[1:, 1:] = invers_report_metrics.iloc[1:, 1:].multiply(100).astype(float).round(2).astype(str) + '%'
invers_report_metrics['date']=invers_report_metrics['date'].astype(str)

In [55]:
report_metrics


,date,likes,views,CTR_%,DAU_FEED,DAU_MESS,DAU_FEEDandMEES,actions,LPU,VPU,message,MPU
0,2022-03-21,145441,662836,21.94,19540,2639,347,808277,7.44,33.92,12900,4.89
1,2022-03-22,143682,660814,21.74,19702,3747,485,804496,7.29,33.54,32259,8.61
2,2022-03-23,120339,589754,20.40,19717,2712,353,710093,6.10,29.91,12879,4.75
3,2022-03-24,92449,471127,19.62,19741,1878,231,563576,4.68,23.87,12900,6.87
4,2022-03-25,96204,474993,20.25,19583,1831,237,571197,4.91,24.26,12901,7.05
5,2022-03-26,137902,650507,21.20,18882,2777,344,788409,7.30,34.45,12899,4.64
6,2022-03-27,160796,739655,21.74,18727,1931,209,900451,8.59,39.50,12900,6.68


In [56]:
value_metrics = invers_report_metrics.iloc[[0,1,-1], 4:].T.values
name_metrics  = report_metrics.iloc[:, 4:].columns.values
col_Labels = ['yesterday','1 day ago','1 week ago']

'DAU_FEED'

In [58]:
# send message to telegram
message = 'Отчет за {v0} по значениям продуктовых метрик по ленте новостей и сообщениям. \nТакже добавлены данные на день и неделю раньше, в процентном отношении от вчерашней даты.'\
        .format(v0 = invers_report_metrics.values[0][0])
message = message + '\nСсылка на основной дашборд <a href=\"https://superset.lab.karpov.courses/superset/dashboard/519/\">Лента новостей и сообщений</a>.'
bot.sendMessage(chat_id=chat_id, text=message, parse_mode = 'HTML')

In [59]:
sns.set(rc={'figure.figsize':(10,8), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12,'axes.titlepad': 30 })
#создаем таблицу
fig, ax = plt.subplots() 

plt.title('Таблица значений продуктовых метрик.')
table = ax.table( 
    cellText = value_metrics,  
    rowLabels = name_metrics,  
    colLabels = col_Labels, 
    rowColours =["steelblue"] * 10,  
    colColours =["steelblue"] * 10, 
    cellLoc ='center',  
    loc="center") 
#bbox = [0.2, 0.4, 0.4, 0.02]
#настройка размера ячеек
ax.axis("off") 
table.auto_set_font_size(False)
table.set_fontsize(15) 
table.auto_set_column_width(col=list(range(len(report_metrics.columns))))

for row in range(1, len(name_metrics)+1):
    cell = table[row, -1]
    cell.set_height(0.1)
for col in range(0, len(col_Labels)):
    for row in range(0, len(name_metrics)+1):
        cell = table[row, col]
        cell.set_height(0.1)
plot_object = io.BytesIO()
plt.savefig(plot_object)
plot_object.seek(0)
plot_object.name = 'Table_metrics.png'
plt.close()
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [68]:
sns.set(rc={'figure.figsize':(32, 30), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12,'axes.titlepad': 30 })
# создание и посылка группы графиков
fig, axs = plt.subplots(4 , 2)
number=0
for i in range(0,len(axs)):
    for j in range(0,len(axs)-2):
        number+=1
        axs[i, j].plot('date', name_metrics[number-1], data = report_metrics)
        axs[i, j].set_title('{v}'.format(v=name_metrics[number-1]))
#добавляет картинку в буфер для посылки        
plot_object = io.BytesIO()
plt.savefig(plot_object)
plot_object.seek(0)
plot_object.name = 'graph_metrics.png'
plt.close()
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [ ]:
ddd

In [60]:
#посылка цсв файла ДФ
file_object = io.StringIO()
report_metrics.to_csv(file_object)
file_object.name = 'report_metrics.csv'
file_object.seek(0)
bot.sendDocument(chat_id=chat_id, document=file_object)